In [1]:
from tensorflow.keras.datasets import mnist
    
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

In [2]:
from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [3]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.SeparableConv2D(32, (3, 3), 
                                 activation='relu', 
                                 input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d (SeparableC (None, 26, 26, 32)        73        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 11, 11, 64)        2400      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 3, 3, 64)          4736      
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [4]:
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy', 
              metrics=['acc'])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 13s 14ms/step - loss: 0.3550 - acc: 0.8935
Epoch 2/5
938/938 [==============================] - 12s 13ms/step - loss: 0.1007 - acc: 0.9683
Epoch 3/5
938/938 [==============================] - 12s 13ms/step - loss: 0.0710 - acc: 0.9773
Epoch 4/5
938/938 [==============================] - 13s 13ms/step - loss: 0.0569 - acc: 0.9821
Epoch 5/5
938/938 [==============================] - 13s 13ms/step - loss: 0.0479 - acc: 0.9853


In [5]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.0451 - acc: 0.9853


[0.045057300478219986, 0.9853000044822693]

In [6]:
import os

base_dir = '../datasets/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(150, 150), 
                                                    batch_size=20, 
                                                    class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir, 
                                                        target_size=(150, 150), 
                                                        batch_size=20,
                                                        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [8]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.SeparableConv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_3 (Separabl (None, 148, 148, 32)      155       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
separable_conv2d_4 (Separabl (None, 72, 72, 64)        2400      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
separable_conv2d_5 (Separabl (None, 34, 34, 128)       8896      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
separable_conv2d_6 (Separabl (None, 15, 15, 128)      

In [9]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.RMSprop(learning_rate=1e-4), 
              metrics=['acc'])

In [10]:
history = model.fit(train_generator, steps_per_epoch=100, epochs=30, 
                              validation_data=validation_generator, 
                              validation_steps=50)

Epoch 1/30
100/100 [==============================] - 16s 155ms/step - loss: 0.6932 - acc: 0.4960 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/30
100/100 [==============================] - 15s 154ms/step - loss: 0.6932 - acc: 0.4970 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 3/30
100/100 [==============================] - 16s 155ms/step - loss: 0.6932 - acc: 0.4990 - val_loss: 0.6929 - val_acc: 0.5000
Epoch 4/30
100/100 [==============================] - 15s 153ms/step - loss: 0.6929 - acc: 0.5095 - val_loss: 0.6928 - val_acc: 0.5000
Epoch 5/30
100/100 [==============================] - 16s 156ms/step - loss: 0.6928 - acc: 0.5085 - val_loss: 0.6924 - val_acc: 0.5100
Epoch 6/30
100/100 [==============================] - 16s 155ms/step - loss: 0.6920 - acc: 0.5250 - val_loss: 0.6915 - val_acc: 0.5160
Epoch 7/30
100/100 [==============================] - 16s 159ms/step - loss: 0.6913 - acc: 0.5295 - val_loss: 0.6887 - val_acc: 0.5910
Epoch 8/30
100/100 [==============================] - 1